In [1]:
import torch
from focal_loss import BinaryFocalLoss, MultiFocalLoss
from torch.nn import CrossEntropyLoss, MSELoss, BCELoss
import torch
import torch.nn as nn
from   torch.nn import functional as F

In [1]:

batch_size, num_class = 16, 2
Loss_Func = MultiFocalLoss(num_class=num_class, gamma=2.0, reduction='mean')

logits = torch.rand(batch_size, num_class, requires_grad=True)  # (batch_size, num_classes)
targets = torch.randint(0, num_class, size=(batch_size, ))  # (batch_size, )

loss = Loss_Func(logits, targets)
loss.backward()

In [37]:
# loss_fct = BinaryFocalLoss()
loss_fct = BCELoss()
batch_size, num_class = 16, 1
pred = torch.rand(batch_size, num_class, requires_grad=True)  # (batch_size, num_classes)
labels = torch.randint(0, num_class+1, size=(batch_size, ))  # (batch_size, )
labels = labels.unsqueeze(1).float()
print(pred.shape, labels.shape)

torch.Size([16, 1]) torch.Size([16, 1])


tensor(1.2944, grad_fn=<BinaryCrossEntropyBackward>)

In [29]:
# pred1 = torch.tensor([[0.5,0.5],[0.3,0.7],[1,0],[1,0]]).float()
# pred1 = torch.tensor([[0.1,0,1,0]]).float()
# pred1 = torch.tensor([[1],[0],[1],[0]]).float()
pred1 = torch.tensor([[0],[1],[0],[1]]).float()
labels1 = torch.tensor([0,1,0,1]).float()
loss_fct = BinaryFocalLoss(alpha=0.25, gamma=2, reduction='mean')
loss1 = loss_fct(pred1, labels1)
loss1

# labels2 = torch.tensor([1,1,1,1]).float()
# print(pred1.shape, labels1.shape, labels2.shape)

torch.Size([4, 1])
tensor([[0.0000],
        [0.0227],
        [0.0000],
        [0.0227]])
torch.Size([4, 1])
tensor([[0.0433],
        [0.0000],
        [0.0433],
        [0.0000]])


tensor(0.0330)

In [25]:
pred1 = torch.tensor([[1,0],[0,1],[1,0],[0,1]]).float()
pred1 = torch.tensor([[0,1],[1,0],[0,1],[1,0]]).float()
labels1 = torch.tensor([0,1,0,1])
print(pred1.shape, labels1.shape)
loss_fct = MultiFocalLoss(alpha=0.25, gamma=2, reduction='mean', num_class=2)
loss1 = loss_fct(pred1, labels1)
loss1

# loss = loss_fct(pred, labels)
# loss

torch.Size([4, 2]) torch.Size([4])


tensor(0.1755)

torch.Size([4, 1])
tensor([[0.0000],
        [0.0227],
        [0.0000],
        [0.0227]])
torch.Size([4, 1])
tensor([[0.1733],
        [0.0000],
        [0.1733],
        [0.0000]])


tensor(0.0980)

In [ ]:
loss_fct = BinaryFocalLoss(alpha=0.25, gamma=2, reduction='mean')
loss1 = loss_fct(pred1, labels1)
loss1

In [12]:
loss2 = loss_fct(pred1, labels2)
loss2

torch.Size([4, 2])
tensor([[0.1733, 0.0227],
        [0.0227, 0.1733],
        [0.0227, 0.1733],
        [0.0227, 0.1733]])
torch.Size([4, 2])
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])


tensor(0.0980)

In [9]:
batch_size = 8
num_class=2
logits = torch.rand(batch_size, num_class, requires_grad=True)  # (batch_size, num_classes)
targets = torch.randint(0, num_class, size=(batch_size, ))  # (batch_size, )
logits.shape, targets.shape

(torch.Size([8, 2]), torch.Size([8]))

In [7]:
x = torch.ones(4,4)
x
x.mean()

tensor(1.)

In [2]:

import torch
import torch.nn.functional as F

input = torch.randn(2, 2)
print(input)

b = torch.softmax(input, dim=1)  # 按列SoftMax,列和为1
print(b)
b = torch.max(input, dim=0)  # 按列Max
print(b)

tensor([[ 0.7491,  0.6891],
        [-1.7619, -1.0933]])
tensor([[0.5150, 0.4850],
        [0.3388, 0.6612]])
torch.return_types.max(
values=tensor([0.7491, 0.6891]),
indices=tensor([0, 0]))


In [5]:
input[:,1].shape

torch.Size([2])

In [16]:
x = torch.tensor([[0.1], [0.2], [0.9]])
x.shape

torch.Size([3, 1])

In [17]:
x = torch.cat([x, 1-x], dim=1)

x.shape

torch.Size([3, 2])

In [22]:
x.int()

tensor([[0, 0],
        [0, 0],
        [0, 0]], dtype=torch.int32)

In [9]:

torch.softmax([])

In [2]:
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer, RobertaModel)

In [3]:
model_name = '../../microsoft/codebert-base'
config = RobertaConfig.from_pretrained(model_name)
config.num_labels = 2
config.feature_size = 14
config.hidden_dropout_prob = 0.1
tokenizer = RobertaTokenizer.from_pretrained(model_name)
special_tokens_dict = {'additional_special_tokens': ["[ADD]", "[DEL]"]}
tokenizer.add_special_tokens(special_tokens_dict)

model = RobertaModel.from_pretrained(model_name, config=config)

model.resize_token_embeddings(len(tokenizer))

Embedding(50267, 768)

In [4]:
emb = model.embeddings.word_embeddings

In [5]:
emb.num_embeddings

50267

In [6]:
emb.embedding_dim

768

In [7]:
x = torch.randint(1,10,(4,10))

y = emb(x)
y.shape

torch.Size([4, 10, 768])

In [8]:
x.shape


torch.Size([4, 10])

In [13]:
class Attention(nn.Module):       #x:[batch, seq_len, hidden_dim*2]
    """
        此注意力的计算步骤：
        1.将输入（包含lstm的所有时刻的状态输出）和w矩阵进行矩阵相乘，然后用tanh压缩到(-1, 1)之间
        2.然后再和矩阵u进行矩阵相乘后，矩阵变为1维，然后进行softmax变化即得到注意力得分。
        3.将输入和此注意力得分线性加权，即相当于将所有时刻的状态进行了一个聚合。
    """
    def __init__(self, hidden_size, need_aggregation=True):
        super().__init__()
        self.need_aggregation = need_aggregation
        # 不双向的话就不用乘2
        self.w = nn.Parameter(torch.Tensor(hidden_size * 2, hidden_size * 2))
        self.u = nn.Parameter(torch.Tensor(hidden_size * 2, 1))
        nn.init.uniform_(self.w, -0.1, 0.1)
        nn.init.uniform_(self.u, -0.1, 0.1)

    def forward(self, x):
        device = x.device
        self.w = self.w.to(device)
        self.u = self.u.to(device)

        u = torch.tanh(torch.matmul(x, self.w))         #[batch, seq_len, hidden_size*2]
        score = torch.matmul(u, self.u)                   #[batch, seq_len, 1]
        att = F.softmax(score, dim=1)
        # 下面操作即线性加权
        scored_x = x * att                              #[batch, seq_len, hidden_size*2]

        # 因为词encoder和句encoder后均带有attention机制，而我需要做的是代码行级缺陷检测，
        # 所以句encoder后我不做聚合，相当于将每个代码行看做一个样本来传入全连接分类。
        if self.need_aggregation:
            context = torch.sum(scored_x, dim=1)                  #[batch, hidden_size*2]
            return context
        else:
            return scored_x



class HAN_MODEL(nn.Module):
    def __init__(self, embedding_layer):
        super().__init__()


        self.hidden_size = 256
        self.num_layers = 1
        self.bidirectional = True

        self.embedding = embedding_layer



        self.lstm1 = nn.LSTM(input_size=self.embedding.embedding_dim,
                            hidden_size=self.hidden_size,
                            num_layers=self.num_layers,
                            bidirectional=self.bidirectional,
                            batch_first=True)
        self.att1 = Attention(self.hidden_size, need_aggregation=True)
        self.lstm2 = nn.LSTM(input_size=self.hidden_size*2,
                            hidden_size=self.hidden_size,
                            num_layers=self.num_layers,
                            bidirectional=self.bidirectional,
                            batch_first=True)
        self.att2 = Attention(self.hidden_size, need_aggregation=True)

        # 代码行级分类输出层，代码有多少行，输出就有多少个神经元
        # self.fc1 = nn.Linear(512, 2)
        self.fc1 = nn.Linear(512, 128)
        self.relu = nn.Tanh()
        self.fc2 = nn.Linear(128, 2)
        # self.softmax = nn.Softmax(dim=-1)


    def forward(self, x):
        # input x : (bs, nus_sentences, nums_words)
        device = x.device
        x = self.embedding(x) # out x : (bs, nus_sentences, nums_words, embedding_dim)
        batch_size, num_sentences, num_words, emb_dim = x.shape

        # 初始化：双向就乘2
        h0_1 = torch.randn(self.num_layers*2, batch_size*num_sentences, self.hidden_size).to(device)
        c0_1 = torch.randn(self.num_layers*2, batch_size*num_sentences, self.hidden_size).to(device)
        h0_2 = torch.randn(self.num_layers*2, batch_size, self.hidden_size).to(device)
        c0_2 = torch.randn(self.num_layers*2, batch_size, self.hidden_size).to(device)

        # 先进入word的处理，将每句话的所有单词的表示通过attention聚合成一个表示
        # 将batch_size, num_sentences两个维度乘起来看成“batch_size”。使用.contiguous()方法首先拷贝了一份张量在内存中的地址，然后将地址按照形状改变后的张量的语义进行排列。
        # touch.view()方法对张量改变“形状”其实并没有改变张量在内存中真正的形状
        x = x.view(batch_size*num_sentences, num_words, emb_dim).contiguous()
        x,(_,_)= self.lstm1(x, (h0_1,c0_1))   # out：batch_size*num_sentences, num_words，hidden_size*2
        x = self.att1(x)   # 线性加权注意力后的输出：batch_size*num_sentences, hidden_size*2

        x = x.view(x.size(0)//num_sentences, num_sentences, self.hidden_size*2).contiguous()
        x,(_,_)= self.lstm2(x, (h0_2,c0_2))   # out：batch_size, num_sentences，hidden_size*2

        x = self.att2(x)   # 线性加权注意力后的输出：batch_size, hidden_size*2
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x

In [14]:

model = HAN_MODEL(embedding_layer=emb)

In [15]:
# batch_size, num_sentences, num_words, embedding_dim
# 类比CV：batch_size, W, H, C
x  = torch.ones(64, 100, 20).long()
out = model(x)
out.shape

torch.Size([64, 2])

In [12]:
x  = torch.ones(64, 100, 20).long()
embedding = nn.Embedding(1000, 300)
out = embedding(x)
print(out.shape)

torch.Size([64, 100, 20, 300])
